In [2]:
import urllib.request
import json
import os
import pandas as pd
import xml.etree.ElementTree as ET

In [4]:
def congress_deco(output_format='json') -> object:
                
                def get_congress(func):
                        
                        print(f'Decorating: {func.__name__}')
                        
                        def wrapper(*args, **kwargs):
                                
                                try:
                                        
                                        url, hdr = func(*args, **kwargs)
                                        
                                        req = urllib.request.Request(url, headers=hdr)
                                        req.get_method = lambda: 'GET'
                                        response = urllib.request.urlopen(req, timeout=5)
                                        
                                        output = response.read()
                                        
                                        if output_format == 'json':
                                                
                                                output = json.loads(output)
                                                
                                        if output_format == 'html':
                                                
                                                root = ET.ElementTree(ET.fromstring(output)).getroot()
                                                
                                                output = [node.text for node in root.findall('.//pre')]
                                        
                                        return output
                                
                                except Exception as e:
                                        
                                        print(e)
                                
                        return wrapper
                
                return get_congress
        
        
@congress_deco(output_format='json')
def get_congress_data(query:str, *args, api_key="6ej9da2smGykG5a2jMYcHY5sWWaznOXPxfZeEz83") -> dict: #os.getenv('US_GOV_api')
                
                hdr = {
                        # specifying requested encoding
                        'Cache-Control': 'no-cache',
                        'charset': 'UTF-8',
                        'X-Api-Key': api_key,
                        'User-Agent': 'Mozilla/5.0'
                        }
                
                BASE_URL = 'https://api.congress.gov/v3/'
                
                query_url = BASE_URL + query
                
                if args: # concatenate variable arguments to url
                                query_url += '&' + '&'.join(args)
                                
                return (query_url, hdr)
        
@congress_deco(output_format='html')
def get_bill_text(query:str, api_key="6ej9da2smGykG5a2jMYcHY5sWWaznOXPxfZeEz83") -> str: #os.getenv('US.GOV_API')
                
                hdr = {
                        # specifying requested encoding
                        'Cache-Control': 'no-cache',
                        'charset': 'UTF-8',
                        'User-Agent': 'Mozilla/5.0'
                        }
                
                return (query, hdr)

Decorating: get_congress_data
Decorating: get_bill_text


In [5]:
def get_congress_data(query:str, *args, api_key= "6ej9da2smGykG5a2jMYcHY5sWWaznOXPxfZeEz83") -> dict: #os.getenv('US_GOV_API')
                
                hdr = {
                        # specifying requested encoding
                        'Cache-Control': 'no-cache',
                        'charset': 'UTF-8',
                        'X-Api-Key': api_key,
                        'User-Agent': 'Mozilla/5.0'
                        }
                
                BASE_URL = 'https://api.congress.gov/v3/'
                
                query_url = BASE_URL + query
                
                if args: # concatenate variable arguments to url
                                query_url += '&' + '&'.join(args)
                                
                return (query_url, hdr)

In [6]:
def get_congress_data(query, *args, api_key="6ej9da2smGykG5a2jMYcHY5sWWaznOXPxfZeEz83"): #os.getenv('US_GOV_api')
                
    try:
        BASE_URL = 'https://api.congress.gov/v3/'
        
        hdr = {
            #specifying requested encoding
            'Cache-Control': 'no-cache',
            'charset': 'UTF-8',
            'X-Api-Key': api_key
            }
                        
        query_url = BASE_URL + query
                        
        if args: # concatenate variable arguments to url
            query_url += '&' + '&'.join(args)
                        
        req = urllib.request.Request(query_url, headers=hdr)
        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)

        string_resp = response.read()
                        
        return json.loads(string_resp)
                        
    except Exception as e:
                        
        print(e)

In [7]:
bills_df = pd.read_csv('data/bills2.csv')
bills_recent = bills_df[bills_df['congress'] > 115]

def get_root_url(url):
    return url.replace('https://api.congress.gov/v3/', '').replace('?format=json', '')

#metadata = get_congress_data(get_root_url(bills_df.iloc[181712]['url']))


#get_congress_data(get_root_url(bills_recent.iloc[1]['url']))
text_url = get_congress_data(
                                get_root_url(row['url']) + '/text')['textVersions'][0]['formats'][0]['url']

NameError: name 'row' is not defined

In [12]:
txt_url = get_congress_data(
    get_root_url(bills_recent.iloc[1]['url']) + '/text')['textVersions'][0]['formats'][0]['url']

#get_bill_text(txt_url)

cosponsors = get_congress_data(get_root_url(metadata['bill']['cosponsors']['url']) + "?", 'limit=250')
                                cosponsor_D = 0
                                cosponsor_R = 0
                                cosponsor_df = pd.DataFrame()
                                for cosponsor in cosponsors['cosponsors']:
                                        cosponsor_party = cosponsor['party']
                                        print(f"cosponsponsor party: {cosponsor_party}")
                                        if cosponsor_party == "D":
                                                cosponsor_D += 1 
                                        if cosponsor_party == "R":
                                                cosponsor_R += 1
                                        print((cosponsor_D, cosponsor_R))
                                        cosposor_name = cosponsor['firstName'] + ' ' + cosponsor['lastName']
                                        cosponsor_party = cosponsor['party']
                                        cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
                                #number = int(output_l[0]['metadata']['bill']['number'])
                                if (cosponsor_D + cosponsor_R)!= 0:
                                        print((cosponsor_D + cosponsor_R))
                                        cosponsor_D_perc = cosponsor_D/(cosponsor_D + cosponsor_R)
                                        cosponsor_R_perc = cosponsor_R/(cosponsor_D + cosponsor_R)

In [20]:
output_l = []
list_df_cosposors = []

for i, row in bills_recent.head(10).iterrows():
                
        try:
                        metadata = get_congress_data(get_root_url(row['url']))
                        # TODO: get infos from metadata dictionary, like sponsors, originChamber, polyArea, title
                        bill_number = metadata['bill']['number']
                        policy_area = metadata['bill']['policyArea']['name']
                        if 'cosponsors' in metadata['bill'].keys():
                                cosponsors = get_congress_data(get_root_url(metadata['bill']['cosponsors']['url']) + "?", 'limit=250')
                                cosponsor_D = 0
                                cosponsor_R = 0
                                cosponsor_df = pd.DataFrame()
                                for cosponsor in cosponsors['cosponsors']:
                                        cosponsor_party = cosponsor['party']
                                        print(f"cosponsponsor party: {cosponsor_party}")
                                        if cosponsor_party == "D":
                                                cosponsor_D += 1 
                                        if cosponsor_party == "R":
                                                cosponsor_R += 1
                                        print((cosponsor_D, cosponsor_R))
                                        cosposor_name = cosponsor['firstName'] + ' ' + cosponsor['lastName']
                                        cosponsor_party = cosponsor['party']
                                        cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
                                #number = int(output_l[0]['metadata']['bill']['number'])
                                if (cosponsor_D + cosponsor_R)!= 0:
                                        print((cosponsor_D + cosponsor_R))
                                        cosponsor_D_perc = cosponsor_D/(cosponsor_D + cosponsor_R)
                                        cosponsor_R_perc = cosponsor_R/(cosponsor_D + cosponsor_R)
                                cosponsor_D = 0
                                cosponsor_R = 0
                                cosponsor_df = pd.DataFrame()
                        else :
                                cosponsor_D_perc = 0
                                cosponsor_R_perc = 0

                        print(cosponsor_D_perc, cosponsor_R_perc)
                        
                        latest_action = metadata['bill']['latestAction']['actionDate']

                        print('Bill number done for df line: ', i)

                        text_url = get_congress_data(
                                get_root_url(row['url']) + '/text')['textVersions'][0]['formats'][0]['url']
                        print('URL text done for df line: ', i)

                        summary = get_congress_data(
                                get_root_url(row['url']) + '/summaries')['summaries'][0]['text']
                        print('Summary done for df line: ', i)

                        # subjects = get_congress_data(
                        #         get_root_url(row['url']) + '/subjects')['subjects']
                        # print('Subjects done for df line: ', i)
                        
                        # related_bills = get_congress_data(
                        #         get_root_url(row['url']) + '/relatedbills')
                        # print('Related bills done for df line: ', i)
                        #TODO: optional, extract information from this. Otherwise 
                        
                        #TODO: extract: amendments, committees, cosponsors(!), titles                        

                        list_df_cosposors.append(cosponsor_df)
                        output_l.append({'bill number': bill_number, 'text_url': text_url, 
                                        'policy_area': policy_area,'summary' : summary, 'latest_action': latest_action,
                                        'cosponsor_D_perc': cosponsor_D_perc, 'cosponsor_R_perc': cosponsor_R_perc})

        except Exception as e:
                print(e)

df_ouput_cosposors = pd.concat(list_df_cosposors)
output_l

cosponsponsor party: R
(0, 1)
cosponsponsor party: R
(0, 2)
cosponsponsor party: D
(1, 2)
cosponsponsor party: R
(1, 3)
cosponsponsor party: R
(1, 4)
cosponsponsor party: D
(2, 4)
cosponsponsor party: R
(2, 5)
cosponsponsor party: D
(3, 5)
cosponsponsor party: R
(3, 6)
cosponsponsor party: D
(4, 6)
cosponsponsor party: R
(4, 7)
cosponsponsor party: R
(4, 8)
cosponsponsor party: R
(4, 9)
13
0.3076923076923077 0.6923076923076923
Bill number done for df line:  181713


/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': co

URL text done for df line:  181713
Summary done for df line:  181713
cosponsponsor party: D
(1, 0)
cosponsponsor party: R
(1, 1)
cosponsponsor party: D
(2, 1)
cosponsponsor party: R
(2, 2)
cosponsponsor party: D
(3, 2)
cosponsponsor party: R
(3, 3)
6
0.5 0.5
Bill number done for df line:  181714


/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': co

URL text done for df line:  181714
Summary done for df line:  181714
0 0
Bill number done for df line:  181715
URL text done for df line:  181715
Summary done for df line:  181715
cosponsponsor party: R
(0, 1)
cosponsponsor party: D
(1, 1)
cosponsponsor party: R
(1, 2)
cosponsponsor party: D
(2, 2)
cosponsponsor party: D
(3, 2)
cosponsponsor party: D
(4, 2)
cosponsponsor party: D
(5, 2)
cosponsponsor party: D
(6, 2)
cosponsponsor party: D
(7, 2)
cosponsponsor party: D
(8, 2)
cosponsponsor party: D
(9, 2)
cosponsponsor party: D
(10, 2)
cosponsponsor party: D
(11, 2)
cosponsponsor party: D
(12, 2)
cosponsponsor party: D
(13, 2)
cosponsponsor party: R
(13, 3)
16
0.8125 0.1875
Bill number done for df line:  181716


...

/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': co

URL text done for df line:  181721
Summary done for df line:  181721
cosponsponsor party: D
(1, 0)
cosponsponsor party: R
(1, 1)
cosponsponsor party: D
(2, 1)
3
0.6666666666666666 0.3333333333333333
Bill number done for df line:  181722


/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/516047199.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': co

URL text done for df line:  181722
Summary done for df line:  181722


[{'bill number': '5472',
  'text_url': 'https://www.congress.gov/116/bills/hr5472/BILLS-116hr5472enr.htm',
  'policy_area': 'Public Lands and Natural Resources',
  'summary': ' <p><b>Jimmy Carter National Historical Park Redesignation Act</b></p> <p>This bill redesignates the Jimmy Carter National Historic Site in Georgia as the Jimmy Carter National Historical Park.</p>',
  'latest_action': '2021-01-13',
  'cosponsor_D_perc': 0.3076923076923077,
  'cosponsor_R_perc': 0.6923076923076923},
 {'bill number': '5126',
  'text_url': 'https://www.congress.gov/116/bills/hr5126/BILLS-116hr5126enr.htm',
  'policy_area': 'Public Lands and Natural Resources',
  'summary': ' <p><b>Direct Enhancement of Snapper Conservation and the Economy through Novel Devices Act of 2019 or the</b> <b>DESCEND Act of 2019</b></p> <p>This bill addresses the use of descending devices to release reef fish in the Gulf of Mexico. (A descending device is an instrument that will release fish at a depth sufficient for the 

In [21]:
for i in output_l:
    print(i["cosponsor_R_perc"], i["cosponsor_D_perc"])
    


0.0 1.0
0.0 1.0
0 0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [18]:

row = bills_recent.iloc[5]

metadata = get_congress_data(get_root_url(row['url']))
                        # TODO: get infos from metadata dictionary, like sponsors, originChamber, polyArea, title
bill_number = metadata['bill']['number']
policy_area = metadata['bill']['policyArea']['name']
if 'cosponsors' in metadata['bill'].keys():
                                cosponsors = get_congress_data(get_root_url(metadata['bill']['cosponsors']['url']) + "?", 'limit=250')
                                cosponsor_D = 0
                                cosponsor_R = 0
                                cosponsor_df = pd.DataFrame()
                                for cosponsor in cosponsors['cosponsors']:
                                        cosponsor_party = cosponsor['party']
                                        print(f"cosponsponsor party: {cosponsor_party}")
                                        if cosponsor_party == "D":
                                                cosponsor_D += 1 
                                        if cosponsor_party == "R":
                                                cosponsor_R += 1
                                        print((cosponsor_D, cosponsor_R))
                                        cosposor_name = cosponsor['firstName'] + ' ' + cosponsor['lastName']
                                        cosponsor_party = cosponsor['party']
                                        cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
                                #number = int(output_l[0]['metadata']['bill']['number'])
                                if (cosponsor_D + cosponsor_R)!= 0:
                                        print((cosponsor_D + cosponsor_R))
                                        cosponsor_D_perc = cosponsor_D/(cosponsor_D + cosponsor_R)
                                        cosponsor_R_perc = cosponsor_R/(cosponsor_D + cosponsor_R)

else:
                                cosponsor_D_perc = 0
                                cosponsor_R_perc = 0
                                cosponsor_df = pd.DataFrame()


print(cosponsor_D_perc, cosponsor_R_perc)


cosponsponsor party: D
(1, 0)
cosponsponsor party: R
(1, 1)
cosponsponsor party: D
(2, 1)
cosponsponsor party: R
(2, 2)
cosponsponsor party: D
(3, 2)
cosponsponsor party: D
(4, 2)
cosponsponsor party: D
(5, 2)
cosponsponsor party: D
(6, 2)
cosponsponsor party: D
(7, 2)
cosponsponsor party: D
(8, 2)
cosponsponsor party: D
(9, 2)
cosponsponsor party: D
(10, 2)
cosponsponsor party: D
(11, 2)
cosponsponsor party: D
(12, 2)
cosponsponsor party: R
(12, 3)
cosponsponsor party: D
(13, 3)
cosponsponsor party: D
(14, 3)
cosponsponsor party: D
(15, 3)
cosponsponsor party: D
(16, 3)
cosponsponsor party: R
(16, 4)
cosponsponsor party: D
(17, 4)
cosponsponsor party: R
(17, 5)
cosponsponsor party: R
(17, 6)
cosponsponsor party: D
(18, 6)
cosponsponsor party: D
(19, 6)
cosponsponsor party: D
(20, 6)
26
0.7692307692307693 0.23076923076923078


/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/26229471.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/26229471.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cosposor_name, 'cosponsor_party': cosponsor_party, 'number': bill_number}, ignore_index=True)
/var/folders/ct/yw5th2lx0d75cqds32p3ws6c0000gn/T/ipykernel_2716/26229471.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cosponsor_df = cosponsor_df.append({'cosponsor_name': cospo

In [69]:
# output_l[0]['metadata']['bill']['cosponsors']['url']
cosponsors = get_congress_data(get_root_url(output_l[0]['metadata']['bill']['cosponsors']['url']) + "?", 'limit=250')



In [120]:
get_congress_data(get_root_url(bills_recent.iloc[11812]['url']))['bill']['latestAction']

{'actionDate': '2019-06-07',
 'text': 'Referred to the Subcommittee on Europe, Eurasia, Energy and the Environment.'}

In [86]:
cosponsor_df = pd.DataFrame()

for cosponsor in cosponsors['cosponsors']:
    consposor_name = cosponsor['firstName'] + ' ' + cosponsor['lastName']
    cosponsor_party = cosponsor['party']
    number = str(output_l[0]['metadata']['bill']['number'])
    cosponsor_df = cosponsor_df.append({'cosponsor_name': consposor_name, 'cosponsor_party': cosponsor_party, 'number': number}, ignore_index=True)

cosponsor_df

,cosponsor_name,cosponsor_party,number
0,Tom Graves,R,5472.0
1,Austin Scott,R,5472.0
2,JOHN LEWIS,D,5472.0
3,Rick Allen,R,5472.0
4,A. Ferguson,R,5472.0
5,Henry Johnson,D,5472.0
6,Barry Loudermilk,R,5472.0
7,David Scott,D,5472.0
8,Earl Carter,R,5472.0
9,Lucy McBath,D,5472.0


In [88]:
outputdf = pd.DataFrame.from_records(output_l)
bills_recent.merge(cosponsor_df, on='number', how="inner")

,Unnamed: 0,congress,latestAction,number,originChamber,originChamberCode,title,type,updateDate,updateDateIncludingText,url,cosponsor_name,cosponsor_party
0,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,Tom Graves,R
1,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,Austin Scott,R
2,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,JOHN LEWIS,D
3,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,Rick Allen,R
4,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,A. Ferguson,R
5,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,Henry Johnson,D
6,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,Barry Loudermilk,R
7,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,David Scott,D
8,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,Earl Carter,R
9,0,116,"{'actionDate': '2021-01-13', 'text': 'Became P...",5472,House,H,Jimmy Carter National Historical Park Redesign...,HR,2022-10-28,2022-10-28T01:46:16Z,https://api.congress.gov/v3/bill/116/hr/5472?f...,Lucy McBath,D


In [80]:
bills_recent.columns

Index(['Unnamed: 0', 'congress', 'latestAction', 'number', 'originChamber',
       'originChamberCode', 'title', 'type', 'updateDate',
       'updateDateIncludingText', 'url'],
      dtype='object')